In [ ]:
#install neccessary packages
!pip install opencv-python
!pip install opencv-contrib-python

This application uses frame differencing and background subtraction techniques to track moving cars. 

First, video feed is loaded and the first frame is stored as prev_frame with a boolean stored as check. A time function is triggered to start counting. A while loop is entered and a second frame is stored as curr_frame with a boolean stored as check.

If check is not True, the loop will break.

The next block of code applies frame differencing and background subtraction techniques. prev_frame and curr_frame are both converted to grayscale and the absolute difference between these frames are generated. The result then passes through a threshold filter and dilation applied to get a stronger difference. A final morphological technique is then used to convolve the result with kernel and dilation/erosion. 

curr_frame is then assigned to prev_frame for iterating through frames.

The following block of code defines the tracking system. Contours are extracted with the result of the previous morphological technique. A for loop is used to loop through all contours with a specified area to create bounding boxes for tracking. A tightly defined region is specified to count cars passing through it.

The next block of code defines a simple algorithm to count number of cars per minute.

Following that, the next block of code creates windows that display the masked video(differencing and background subtraction) and the video with tracking and other text.

At the end, the if statement allows the key 'q' to be pressed in order to exit the application and close the windows.

In [1]:
import cv2 
import numpy as np
import time
import math

#load video
video = cv2.VideoCapture("Traffic_Laramie_2.mp4")
#read first frame
check, prev_frame = video.read()

cars = 0
car=0
cpm=0

#start timer
start = time.time()

while video.isOpened():
    #read second frame
    check, curr_frame = video.read() #reads next frame
    frame = curr_frame.copy()
    
    #check if all the frames have been read
    if not check:
        break
        
    #convert frames into grayscale
    gray_prev= cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray_curr = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    #extract absolute difference between frames
    abs_diff = cv2.absdiff(gray_prev, gray_curr)
    #apply threshold 
    _, thres = cv2.threshold(abs_diff,10,255, cv2.THRESH_BINARY)
    #create kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    #convolve with dilation
    dilate = cv2.dilate(thres, np.ones((5,5)))
    #convolve with kernel and dilation/erosion 
    closing = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)
    
    #assign second frame to first frame for iteration to the end
    prev_frame = curr_frame

    #extract contours
    contours, _ = cv2.findContours(closing, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #loop to track and display bounding boxes
    for contour in contours:
        bounding_area = cv2.contourArea(contour)
        #only create bounding boxes for countours with above certain area
        if bounding_area > 5150:
            x,y,w,h = cv2.boundingRect(contour)
            #restrict tracking and bounding boxes to only main street
            if y > 180:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #cars counter
            if x > 497 and x < 505 and y <420:
                cars +=1
     
    #get time elapsed
    seconds = time.time()
    time_elapsed = int(seconds)-int(start)
    minute = int(math.floor(time_elapsed/60))
    #count cars per minute
    if minute == 0:
        cpm = cars
    if minute > 1:
        cpm = cars / minute


    #diplay videos
    cv2.imshow('Mask', closing) #masking video
    cv2.putText(frame, 'Seconds : {}'.format(time_elapsed), (800,50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255),2)
    cv2.putText(frame, 'Total cars : {}'.format(cars), (300,230), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255),2)
    cv2.putText(frame, 'Cars per minute: {}'.format(cpm), (600,230), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255),2)
    cv2.putText(frame, 'Main Street', (20,250), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),2)
    cv2.rectangle(frame, (0, 260), (1040, 600), (0,0,255), 2)
    cv2.imshow('Input', frame) #input video

    #to quit press key "q", to change to normal speed replace change value to 30
    if cv2.waitKey(30) & 0xFF == ord("q"):
        break

#close windows
video.release()
cv2.destroyAllWindows()